In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import preprocess_text
import random
import tqdm

from models.transformer import Transformer
from data.collate_fn import collate_fn
from train.lr_scheduler import NoamLR
from train.loss import loss_function
from train.train import train_loop

import torch
from torch.optim import Adam

from data.dataset import AihubTranslationDataset
from torch.utils.data import DataLoader

2.4.0


/home/masang/anaconda3/envs/torchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 데이터 전처리

In [2]:
# Dataset
train_ds = AihubTranslationDataset(
    csv_path="ai_hub_dataset/train_filtered.csv",
    preprocess_fn=preprocess_text,
    max_len=60,
    add_special_tokens=True,
)

In [3]:
print(train_ds[1972]["tgt_ids"])

tensor([  101,  2009,  2003,  2036,  2691,  2005,  1037,  7968,  2158,  2000,
         2022,  8040, 27479,  2030,  2191, 13219,  6567,  1012,   102])


In [4]:
train_loader = DataLoader(
    dataset=train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn
)

#### 모델 학습

In [5]:
epochs = 10
batch_size = 32

In [6]:
# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 생성 및 디바이스 이동
model = Transformer(
    src_vocab_size=32000,  # "klue/bert-base" tokenizer vocab_size
    tgt_vocab_size=30522,  # "bert-base-uncased" tokenizer vocab_size
    src_len=60,  # 인코더 입력 길이 제한
    tgt_len=60,  # 디코더 입력 길이 제한
    d_model=512,
    d_ff=2048,
    n_heads=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    dropout=0.3,
).to(device)

In [7]:
# optimizer & lr scheduler 정의
optimizer = Adam(model.parameters(), lr=1, betas=(0.9, 0.98), eps=1e-9)
scheduler = NoamLR(optimizer, d_model=512, warmup_steps=4000)

In [8]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_loader, model, loss_function, optimizer, scheduler, device)
print("Done!")

Epoch 1
-------------------------------


ValueError: Expected `device_type` of type `str`, got: `<class 'torch.device'>`